In [1]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
trips = pq.read_table('./Data/yellow_tripdata_2021-06.parquet')
trips = trips.to_pandas()
trips.dropna(inplace = True)

# data cleaning
trips = trips.query('passenger_count > 0.0 and trip_distance > 0.0 and trip_distance < 100 and total_amount > 0.0')
trips = trips.query('tpep_pickup_datetime >= "2021-06-01" and tpep_pickup_datetime < "2021-07-01"')

In [3]:
# add boroughs
taxi_df = pd.read_csv("./Data/taxi_zone_lookup.csv", usecols = [0,1,2])

PU_map = taxi_df.rename(columns = {'LocationID':'PULocationID', 'Borough':'PU_borough', 'Zone':'PU_zone'})
DO_map = taxi_df.rename(columns = {'LocationID':'DOLocationID', 'Borough':'DO_borough', 'Zone':'DO_zone'})
trips = trips.join(PU_map.set_index('PULocationID'), on='PULocationID')
trips = trips.join(DO_map.set_index('DOLocationID'), on='DOLocationID')
trips = trips.query('PU_borough == "Manhattan" and DO_borough == "Manhattan"')
trips = trips.drop(['PU_borough', 'DO_borough', 'PULocationID', 'DOLocationID'], axis = 1)

trips['From_To'] = trips['PU_zone'] + " - " + trips['DO_zone']

# add tip percentage and buckets
trips["tip_pct"] = trips["tip_amount"] / trips["total_amount"]
trips["tip_bin"] = pd.cut(trips.tip_pct, 
                       bins = [0.0, 0.10, 0.2, 0.5, 1.0], 
                       labels = ["[0%, 10%]", "(10%, 20%]", "(20%, 50%]", "(50%, 100%]"], 
                       include_lowest = True)


In [4]:
# add weather data
trips['Date'] = pd.to_datetime(trips['tpep_pickup_datetime']).dt.date

weather = pd.read_csv("./Data/2021NYCWeather.csv", 
                      parse_dates=[0],
                      usecols = [0, 2, 5, 8, 11, 14, 16], 
                      dtype = {'avg_temp':'float64', 
                               'avg_dew_pt':'float64', 
                               'avg_humidity':'float64',
                               'avg_wind_speed':'float64',
                               'avg_pressure':'float64',
                               'total_precipitation':'float64'})
weather['Date'] = pd.to_datetime(weather['Date']).dt.date

trips = trips.join(weather.set_index('Date'), on = "Date")
trips = trips.drop(['Date'], axis = 1)

/var/folders/x6/q38n8vvn4438v5128825zdl40000gn/T/ipykernel_4977/3135072809.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather = pd.read_csv("./Data/2021NYCWeather.csv",


In [5]:
# write to parquet file
trips.iloc[:, [0, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 20, 21, 12, 13, 14, 15, 16, 22, 23, 24, 25, 26, 27]].to_parquet('./Data/cleaned_taxi_data.parquet')

In [6]:
# how to open parquet file
temp = pq.read_table('./Data/cleaned_taxi_data.parquet')
temp = temp.to_pandas()

temp = temp.astype({'VendorID':'category',
                    'RatecodeID':'category', 
                    'store_and_fwd_flag':'bool',
                    'payment_type':'category', 
                    'passenger_count':'int64', 
                    'PU_zone':'string',
                    'DO_zone':'string',
                    'From_To':'string'
                   })

temp.dtypes

VendorID                       category
PU_zone                  string[python]
DO_zone                  string[python]
From_To                  string[python]
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                     category
store_and_fwd_flag                 bool
payment_type                   category
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tip_pct                         float64
tip_bin                        category
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
avg_temp                        float64
avg_dew_pt                      float64
avg_humidity                    float64
